In [5]:
from pathlib import Path
import pandas as pd
import requests
import io
import zipfile

In [ ]:
import requests
from pathlib import Path
import zipfile
import io
import pandas as pd

def fetch_raw_data(year: int, month: int) -> str:
    # Construct the correct URL
    ext = "csv.zip" if month <= 4 else "zip"
    url = f"https://s3.amazonaws.com/tripdata/{year}{month:02}-citibike-tripdata.{ext}"

    # Download the ZIP file
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"{url} is not available")

    # Prepare path to save final CSV
    raw_dir = Path("..") / "data" / "raw"
    raw_dir.mkdir(parents=True, exist_ok=True)
    output_file = raw_dir / f"rides_{year}_{month:02}.csv"

    # Process ZIP and read all CSV files
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        csv_files = [f for f in z.namelist() if f.endswith('.csv')]
        df_list = []

        for file in csv_files:
            df = pd.read_csv(z.open(file), encoding='latin1',low_memory=False)
            
            # Optional: Drop unnamed or empty extra columns
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            df_list.append(df)

        combined_df = pd.concat(df_list, ignore_index=True)

        # Save combined CSV
        combined_df.to_csv(output_file, index=False)

    print(f"Successfully saved combined CSV to: {output_file}")
    return str(output_file)


In [15]:
fetch_raw_data(2024,5)

C:\Users\HP\AppData\Local\Temp\ipykernel_21692\1699710808.py:28: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(z.open(file), encoding='latin1')
C:\Users\HP\AppData\Local\Temp\ipykernel_21692\1699710808.py:28: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(z.open(file), encoding='latin1')
C:\Users\HP\AppData\Local\Temp\ipykernel_21692\1699710808.py:28: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(z.open(file), encoding='latin1')
C:\Users\HP\AppData\Local\Temp\ipykernel_21692\1699710808.py:28: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(z.open(file), encoding='latin1')
C:\Users\HP\AppData\Local\Temp\ipykernel_21692\1699710808.py:28: DtypeWarning: Columns (5,7) have mixed types. Specify dtype o

Successfully saved combined CSV to: ..\data\raw\rides_2024_05.csv


'..\\data\\raw\\rides_2024_05.csv'